In [1]:
# 기본 작업 경로 설정

import os
notebook_path = os.path.abspath("../readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


### https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [2]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# 데이터셋 경로와 형식 설정
register_coco_instances("face_data_set", {}, "data/ssd_rcnn_face/annotations.json", "data/yolo_data/train/")
register_coco_instances("face_data_set_valid", {}, "data/ssd_rcnn_face/annotations_val.json", "data/yolo_data/val/")

# 메타데이터를 확인.
metadata = MetadataCatalog.get("face_data_set")
dataset_dicts = DatasetCatalog.get("face_data_set")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.config import CfgNode as CN
from detectron2.utils.logger import setup_logger
from detectron2.model_zoo import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
'''
각 하이퍼 파라미터 설명
cfg.DATASETS.TRAIN : 학습에 사용할 데이터셋을 설정.
cfg.DATASETS.TEST : 테스트에 사용할 데이터셋을 설정.
cfg.DATALOADER.NUM_WORKERS : 데이터로더의 워커 수를 설정.
cfg.MODEL.WEIGHTS : 모델의 가중치를 설정.
cfg.SOLVER.IMS_PER_BATCH : 배치 크기를 설정.
cfg.SOLVER.BASE_LR : 학습률을 설정.
cfg.SOLVER.MAX_ITER : 최대 반복 횟수를 설정.
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE : 이미지 당 ROI 배치 크기를 설정.
cfg.MODEL.ROI_HEADS.NUM_CLASSES : 클래스 수를 설정. (배경 클래스 포함 하지 않음)
cfg.OUTPUT_DIR : 모델의 저장 경로를 설정.

추가 하이퍼 파라미터
cfg.SOLVER.CHECKPOINT_PERIOD : 체크포인트 저장 주기를 설정.
cfg.TEST.EVAL_PERIOD : 평가 주기를 설정.
'''
class CustomTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.cfg = cfg

    def train(self):
        super().train()
        # Add evaluation after each epoch
        for epoch in range(self.start_iter, self.max_iter + 1):
            if epoch % self.cfg.TEST.EVAL_PERIOD == 0:
                self.do_test()
            if epoch % self.cfg.SOLVER.CHECKPOINT_PERIOD == 0:
                self.checkpointer.save(f"model_{epoch}")

    def do_test(self):
        # Perform evaluation
        evaluator = COCOEvaluator(self.cfg.DATASETS.TEST[0], output_dir=self.cfg.OUTPUT_DIR)
        val_loader = build_detection_test_loader(self.cfg, self.cfg.DATASETS.TEST[0])
        results = inference_on_dataset(self.model, val_loader, evaluator)
        print(results)


cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# 다양한 백본 설정 가능
# cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
# cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
# cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
# ... 등 등

cfg.DATASETS.TRAIN = ("face_data_set",)
cfg.DATASETS.TEST = ("face_data_set_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 100

# 입력 사이즈 설정
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512

# 간단한 증강
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.RANDOM_ROTATION = 30  # 최대 30도 회전
cfg.INPUT.CROP = CN({"ENABLED": True, "TYPE": "relative_range", "SIZE": [0.8, 0.8]})

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 사용자 클래스 수로 설정
cfg.OUTPUT_DIR = "./models/faster_rcnn_R_50_FPN_3x"

# 검증 평가 주기
cfg.TEST.EVAL_PERIOD = 100  # 매 100 iterations마다 평가

# 체크포인트 저장 주기
cfg.SOLVER.CHECKPOINT_PERIOD = 500  # 매 500 iterations마다 체크포인트 저장

# 학습률 스케줄러 설정
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"  # 또는 "WarmupMultiStepLR"
cfg.SOLVER.WARMUP_ITERS = 500  # Warmup iterations 설정

setup_logger()

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()



In [13]:
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_dir=None):
        return COCOEvaluator(dataset_name, cfg, True, output_dir)

    def train(self):
        super().train()
        # Add evaluation after each epoch
        for epoch in range(self.start_iter, self.max_iter + 1):
            if epoch % self.cfg.TEST.EVAL_PERIOD == 0:
                self.do_test()
            if epoch % self.cfg.SOLVER.CHECKPOINT_PERIOD == 0:
                self.checkpointer.save(f"model_{epoch}")

    def do_test(self):
        # Perform evaluation
        evaluator = self.build_evaluator(self.cfg, self.cfg.DATASETS.TEST[0], output_dir=self.cfg.OUTPUT_DIR)
        val_loader = build_detection_test_loader(self.cfg, self.cfg.DATASETS.TEST[0])
        results = inference_on_dataset(self.model, val_loader, evaluator)
        print(results)

# Set up configuration as before
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("face_data_set",)
cfg.DATASETS.TEST = ("face_data_set_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 100
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.RANDOM_ROTATION = 30
cfg.INPUT.CROP = CN({"ENABLED": True, "TYPE": "relative_range", "SIZE": [0.8, 0.8]})
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.OUTPUT_DIR = "./models/faster_rcnn_R_50_FPN_3x"
cfg.TEST.EVAL_PERIOD = 10
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = 500

setup_logger()
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[09/06 14:32:48 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[09/06 14:33:19 d2.engine.train_loop]: Starting training from iteration 0
[09/06 14:33:46 d2.data.datasets.coco]: Loaded 1199 images in COCO format from data/ssd_rcnn_face/annotations_val.json
[09/06 14:33:46 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(512, 512), max_size=512, sample_style='choice')]
[09/06 14:33:46 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/06 14:33:46 d2.data.common]: Serializing 1199 elements to byte tensors and concatenating them all ...
[09/06 14:33:46 d2.data.common]: Serialized dataset takes 0.36 MiB
WARNING [09/06 14:33:46 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[09/06 14:33:46 d2.evaluation.evaluator]: Start inference on 1199 batches
[09/06 14:33:48 d2.evaluation.evaluator]: Inference done 11/1199. Dataloading: 0.0483 s/iter. 

KeyboardInterrupt: 

In [7]:
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.config import CfgNode as CN
import json
import os

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_dir=None):
        return COCOEvaluator(dataset_name, cfg, True, output_dir)
    
    def train(self):
        self.results = {"train": [], "test": []}
        super().train()
        
        # Add evaluation after each epoch
        for epoch in range(0, self.cfg.SOLVER.MAX_ITER + 1, self.cfg.TEST.EVAL_PERIOD):
            self.do_test(epoch)
            if epoch % self.cfg.SOLVER.CHECKPOINT_PERIOD == 0:
                self.checkpointer.save(f"model_{epoch}")

    def do_test(self, epoch):
        try:
            # Perform evaluation
            evaluator = self.build_evaluator(self.cfg, self.cfg.DATASETS.TEST[0], output_dir=self.cfg.OUTPUT_DIR)
            val_loader = build_detection_test_loader(self.cfg, self.cfg.DATASETS.TEST[0])
            results = inference_on_dataset(self.model, val_loader, evaluator)
            self.results["test"].append({"epoch": epoch, "results": results})
            print("Evaluation results:", results)
    
            # Save the results as JSON
            self.save_results_as_json()
        except Exception as e:
            print(f"Error during evaluation at epoch {epoch}: {e}")
    
    def save_results_as_json(self):
        try:
            output_file_path = os.path.join(self.cfg.OUTPUT_DIR, "train_test_results.json")
            print(f"Saving results to: {output_file_path}")
            with open(output_file_path, 'w') as f:
                json.dump(self.results, f, indent=2)
        except Exception as e:
            print(f"Error saving results as JSON: {e}")


# Set up configuration
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("face_data_set",)
cfg.DATASETS.TEST = ("face_data_set_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 100
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.RANDOM_ROTATION = 30
cfg.INPUT.CROP = CN({"ENABLED": True, "TYPE": "relative_range", "SIZE": [0.8, 0.8]})
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.OUTPUT_DIR = "./models/faster_rcnn_R_50_FPN_3x"
cfg.TEST.EVAL_PERIOD = 1000
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = 500

setup_logger()
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[09/06 15:09:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[09/06 15:09:02 d2.engine.train_loop]: Starting training from iteration 0
[09/06 15:09:15 d2.data.datasets.coco]: Loaded 1199 images in COCO format from data/ssd_rcnn_face/annotations_val.json
[09/06 15:09:15 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(512, 512), max_size=512, sample_style='choice')]
[09/06 15:09:15 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/06 15:09:15 d2.data.common]: Serializing 1199 elements to byte tensors and concatenating them all ...
[09/06 15:09:15 d2.data.common]: Serialized dataset takes 0.36 MiB
WARNING [09/06 15:09:15 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[09/06 15:09:15 d2.evaluation.evaluator]: Start inference on 1199 batches
[09/06 15:09:16 d2.evaluation.evaluator]: Inference done 11/1199. Dataloading: 0.0409 s/iter. 

KeyboardInterrupt: 